# Агрегаты и оконные функции, сортировка

### Агрегаты

* агрегирование - это получение итогового значения по некой группе
* "тип" итогового значения обычно определяется агрегирующей функцией (sum, max, ...)
* группировать можно разными способами
    * по всему датафрейму (нет группировки)
    * по его колонкам датафрейма
    * с использованием "оконных" функций

### Агрегирующие функции

Их много, см. документацию, основные - min(), max(), sum(), count(), countDistinct().

Агрегирующие функции - это трансформации.

### Группировка

* `df.select(sum("column"))` - по всему датафрейма
* `groupBy()` - создает `RelationalGroupedDataset`, который потом агрегируем
* оконная функция задает `window`, которое потом используется агрегирующей функцией

Оконные функции - мощный и универсальный механизм, см. материалы, там приведеные примеры его применения.

### Сортировка

* два метода (синонимы) `sort()` и `orderBy()`
* сортируем по значению колонки/колонок
* задавать колонки можно именами или строковым выражением
* сортировать можно по возрастанию или убыванию


### Практикуемся

In [5]:
import os
from pyspark.sql import SparkSession
from pyspark.sql import functions as f
os.environ["SPARK_HOME"] = "/home/mk/mk_win/projects/SparkEdu/lib/python3.5/site-packages/pyspark"
os.environ["PYSPARK_PYTHON"] = "/usr/bin/python3"
os.environ["PYSPARK_DRIVER_PYTHON"] = "python3"
os.environ["PYSPARK_SUBMIT_ARGS"] = "pyspark-shell"

In [2]:
spark = SparkSession.builder.master("local").appName("spark_aggr").getOrCreate()

In [3]:
cdf = spark.read.format("csv") \
    .option("mode", "FAILFAST") \
    .option("inferSchema", "true") \
    .option("header","true") \
    .option("path", "data/countries of the world.csv") \
    .load()

**Без группировки**

Агрегируем по всему датафрейму

In [7]:
cdf.select(f.max("Population")).show()

+---------------+
|max(Population)|
+---------------+
|     1313973713|
+---------------+



**groupBy**

Посмотрим агрегат в разбивке по колонке (используем groupBy())

In [9]:
cdf.groupBy("Region").max("Population").show()

+--------------------+---------------+
|              Region|max(Population)|
+--------------------+---------------+
|BALTICS          ...|        3585906|
|C.W. OF IND. STATES |      142893540|
|ASIA (EX. NEAR EA...|     1313973713|
|WESTERN EUROPE   ...|       82422299|
|NORTHERN AMERICA ...|      298444215|
|NEAR EAST        ...|       70413958|
|EASTERN EUROPE   ...|       38536869|
|OCEANIA          ...|       20264082|
|SUB-SAHARAN AFRIC...|      131859731|
|NORTHERN AFRICA  ...|       78887007|
|LATIN AMER. & CAR...|      188078227|
+--------------------+---------------+



**оконная функция**

Сделаем то же самое, но с использованием оконной функции.

In [22]:
from pyspark.sql.window import Window
windowSpec = Window.partitionBy("Region")
cdf.select("Region",f.max("Population").over(windowSpec).alias("MaxPopulation")).distinct().show()

+--------------------+-------------+
|              Region|MaxPopulation|
+--------------------+-------------+
|BALTICS          ...|      3585906|
|C.W. OF IND. STATES |    142893540|
|ASIA (EX. NEAR EA...|   1313973713|
|WESTERN EUROPE   ...|     82422299|
|NORTHERN AMERICA ...|    298444215|
|NEAR EAST        ...|     70413958|
|EASTERN EUROPE   ...|     38536869|
|OCEANIA          ...|     20264082|
|SUB-SAHARAN AFRIC...|    131859731|
|NORTHERN AFRICA  ...|     78887007|
|LATIN AMER. & CAR...|    188078227|
+--------------------+-------------+



**Сортировка**

Упорядочим предыдущий результат по убыванию максимального населения

In [23]:
cdf.select("Region",f.max("Population").over(windowSpec).alias("MaxPopulation")).distinct().sort(f.desc("MaxPopulation")).show()

+--------------------+-------------+
|              Region|MaxPopulation|
+--------------------+-------------+
|ASIA (EX. NEAR EA...|   1313973713|
|NORTHERN AMERICA ...|    298444215|
|LATIN AMER. & CAR...|    188078227|
|C.W. OF IND. STATES |    142893540|
|SUB-SAHARAN AFRIC...|    131859731|
|WESTERN EUROPE   ...|     82422299|
|NORTHERN AFRICA  ...|     78887007|
|NEAR EAST        ...|     70413958|
|EASTERN EUROPE   ...|     38536869|
|OCEANIA          ...|     20264082|
|BALTICS          ...|      3585906|
+--------------------+-------------+



**Минимум или максимум**

Завершающий пример - чуть усложним: найдем страны с минимальным и максимальным населением в регионах

Комментарии:

* оконная функция просто группирует по региону
* добавляем колонки с минимальным и максимальным населением по региону (`minp, maxp`)
* оставляем в датафрейме только страны, население которых самое маленькое или большое (используем expr)
* добавляем колонку `which`, в которую записываем - какая это страна (с минимальным или максимальным населением, используем функции `when().otherwise()`
* удалим неинтересные нам колонки (`select()`)
* упорядочим - покажем сначала самые большие страны, потом - самые маленькие (по убыванию населения)

In [54]:
wr = Window.partitionBy('Region')
cdf.withColumn('minp', f.min('Population').over(wr))\
    .withColumn('maxp', f.max('Population').over(wr)) \
    .where((f.expr('Population==minp or Population==maxp') ) ) \
    .withColumn("which",f.when(f.expr("Population==minp"),'MIN').otherwise('MAX')) \
    .select("Country","Region","Population","which") \
    .sort("which",f.desc("Population")) \
    .show(100)


+--------------------+--------------------+----------+-----+
|             Country|              Region|Population|which|
+--------------------+--------------------+----------+-----+
|              China |ASIA (EX. NEAR EA...|1313973713|  MAX|
|      United States |NORTHERN AMERICA ...| 298444215|  MAX|
|             Brazil |LATIN AMER. & CAR...| 188078227|  MAX|
|             Russia |C.W. OF IND. STATES | 142893540|  MAX|
|            Nigeria |SUB-SAHARAN AFRIC...| 131859731|  MAX|
|            Germany |WESTERN EUROPE   ...|  82422299|  MAX|
|              Egypt |NORTHERN AFRICA  ...|  78887007|  MAX|
|             Turkey |NEAR EAST        ...|  70413958|  MAX|
|             Poland |EASTERN EUROPE   ...|  38536869|  MAX|
|          Australia |OCEANIA          ...|  20264082|  MAX|
|          Lithuania |BALTICS          ...|   3585906|  MAX|
|            Armenia |C.W. OF IND. STATES |   2976372|  MIN|
|           Slovenia |EASTERN EUROPE   ...|   2010347|  MIN|
|            Estonia |BA

In [ ]:
spark.stop()